In [ ]:
import json
import pandas as pd
import requests
import time
import datetime

In [2]:
wallet_address = 'bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx'
api_url = 'https://blockchain.info'
api_end_point = 'rawaddr'

In [3]:
url = f'{api_url}/{api_end_point}/{wallet_address}'
response = requests.get(url)
response

<Response [200]>

In [4]:
response_body = response.json()
TX_COUNT = response_body['n_tx']
TX_COUNT

1653

In [5]:
ITERATIONS = int(TX_COUNT / 100) 
ITERATIONS

16

In [9]:
transactions_df = None
i = 0
offset = 0
url = f'{api_url}/{api_end_point}/{wallet_address}?format=json&offset={offset}'
wallet = pd.read_json(url)
transactions_df = wallet['txs']
transactions_df.to_json(f'tx{offset}.json')
value = 30
while i <= ITERATIONS:
    offset = offset + 100
    print(offset)
    url = f'{api_url}/{api_end_point}/{wallet_address}?format=json&offset={offset}'
    print(f'requesting: {url}')
    wallet = pd.read_json(url)
    transactions_df = wallet['txs']
    transactions_df.to_json(f'tx{offset}.json')
    i = i + 1
    time.sleep(value)


requesting: https://blockchain.info/rawaddr/bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx?format=json&offset=1600


In [6]:
i = 0
ITERATIONS = 16
out_transactions_data = []
in_transactions_data = []
count = 0
while i <= ITERATIONS:
    f_name = f'tx{i*100}.json'
    print(f'opeing file {f_name}')
    file = open(f_name, 'r')
    txs = json.load(file)
    address = 'bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx'
    
    for index in txs:
        addresses = set()
        tx = txs[index]
        tx_hash = tx['hash']
        tx_time = datetime.datetime.fromtimestamp(tx['time'])
        inputs = tx['inputs']
        outs = tx['out']
        fees = float(tx['fee']) * 0.00000001
        is_out_tx = False
        tx_amount = 0
        count += 1
        for input in inputs:
            prev = input['prev_out']
            if prev['addr'] == address:
                is_out_tx= True
            else:
                addresses.add(prev['addr'])

        if is_out_tx:
            out_tx = outs[0]
            tx_amount = float(out_tx['value']) * 0.00000001
            out_tx_address = out_tx['addr']
            out_transactions_data.append([tx_hash, address, tx_amount, out_tx_address, tx_time])
        else:
            for out_tx in outs:
                if out_tx['addr'] == address:
                    tx_amount = float(out_tx['value']) * 0.00000001
                    out_tx_address = out_tx['addr']
                    in_transactions_data.append([tx_hash, '##'.join(str(s) for s in addresses), tx_amount, out_tx_address, tx_time])
    i = i + 1
print(count)

opeing file tx0.json
opeing file tx100.json
opeing file tx200.json
opeing file tx300.json
opeing file tx400.json
opeing file tx500.json
opeing file tx600.json
opeing file tx700.json
opeing file tx800.json
opeing file tx900.json
opeing file tx1000.json
opeing file tx1100.json
opeing file tx1200.json
opeing file tx1300.json
opeing file tx1400.json
opeing file tx1500.json
opeing file tx1600.json
1653


In [7]:
out_df = pd.DataFrame(out_transactions_data, columns=['tx_hash', 'account_address', 'amount', 'to_address', 'tx_time'])
in_df = pd.DataFrame(in_transactions_data, columns=['tx_hash', 'account_address', 'amount', 'to_address', 'tx_time'])

In [8]:
out_df

,tx_hash,account_address,amount,to_address,tx_time
0,53392049d344b4f016d3560dece5765d43e42bdfb181ff...,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.001008,bc1qlx09ds46r0l54t0aawn79x5t9mef2sj9dk2ce7,2021-12-25 05:17:20
1,52b4599f00e6c75d6c454f3ae6c29097dbe3eba8a9d248...,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.000443,3DscuRBWfD927YKHnd17pzAdeRSXApaeM6,2021-12-12 15:51:53
2,91a51216ce148b2a7636db67e1725ea53d1b133947e11b...,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.009322,bc1qs3uxa3686702x4mpx2tt2y99s65n706yun5ey3,2021-12-08 07:27:00
3,f8293c5e6ed4febe48294eaebcce91a0721cd2680e9118...,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.009322,bc1qs3uxa3686702x4mpx2tt2y99s65n706yun5ey3,2021-12-08 07:27:00
4,dbddbf8e66ab74dbe53164fa77d9cb770dbaddf4e92e9b...,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.009322,bc1qs3uxa3686702x4mpx2tt2y99s65n706yun5ey3,2021-12-08 07:27:00
...,...,...,...,...,...
473,b36ef2b88acf18e137ebc379a6b90ce9665bf55f2ce5d5...,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.000900,1B5T7xJTGU2vHyw7E8kN5LeWXdrarCMVr6,2020-11-25 03:32:36
474,ca5d2566c7a28e0b1a2162f4e108dc1a9fc417d6ab7322...,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.001600,1B5T7xJTGU2vHyw7E8kN5LeWXdrarCMVr6,2020-11-25 02:29:47
475,8cf6d9281189fd13bcf87e15c6b45d9d044715db9d64a7...,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.000900,bc1qppmweh4rn0aj9calktz8mhuzf2v5kmdt0c5fek,2020-11-24 18:23:19
476,399dbfcf6e1938089851fdb37728b5d2a009415d4c9ce6...,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,0.001600,1B5T7xJTGU2vHyw7E8kN5LeWXdrarCMVr6,2020-11-24 08:23:30


In [9]:
in_df

,tx_hash,account_address,amount,to_address,tx_time
0,3a56b67e401774a84c6fb9612b899a07c963f1ae91ae52...,bc1qz6z4z98mdk6fdawfhj28ype2t25whzr0330zth##bc...,0.000500,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,2021-10-29 17:01:53
1,56df5cc65ea8a8ddf1dfa9476994e17d3b2f8c75d896a3...,bc1qmks7xceg8ve433ed8cdex58aeepf2cy62nwrsw,0.007984,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,2021-10-06 04:46:14
2,680dc70c28396a5a195a5cc0e2acf9175204eb30c5b92b...,bc1q6aa0gx03aah60xmsl8fxtjm9wcrululgj732ak##bc...,0.084961,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,2021-09-22 23:09:59
3,176f8be85a53ea439ea802ee26412811e01b88deff623c...,bc1q4270k3ljzk0hzquc293ypg0g4jerw2d0kwslwv,0.002431,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,2021-08-17 22:56:25
4,de4ac27c790c46eb00ce9522b3dc372cfdb730a6075e58...,bc1q2djquweh5cpg02rjhscheu0z5z636pfmng2845,0.000138,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,2021-08-17 22:56:25
...,...,...,...,...,...
1170,db6266cc11e36f65a6aa5621a80be2c546c4460f8230a1...,bc1qd3c87cfgcau4rtamff35e9dqgxdc4a6scpl7x4,0.002808,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,2020-11-22 18:54:23
1171,1db03d05ad0ca70218cde9a8220debc59786fdfc46decc...,bc1qfhjx4yujtqcu4kgw6sgvgh26nnvyw7zvel44xf,0.000457,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,2020-11-22 18:54:23
1172,5dc1c66c4b24629e68e16790b9528861d7af77fca34187...,bc1qgrfvtmeklfc69ect58hm2ymsutu7mc0rkpva8l,0.006312,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,2020-11-22 18:54:23
1173,47644b3d3e95051e570e5352af599b3d87c39629cba034...,bc1q4dt433mufpujfd92lc98p4m63gjpwxutduz7ea,0.000708,bc1q4fdqv7z8mqk73rtgzk7qdxnzw675320gjar6xx,2020-11-22 18:54:23


In [10]:
out_df.to_csv('btc_out_trxs.csv', sep=';')
in_df.to_csv('btc_in_trxs.csv', sep=';')

In [12]:
n_in_trxs = len(in_df)
n_out_trxs = len(out_df)
total_in = in_df['amount'].sum()
total_out = out_df['amount'].sum()
total_out_fees = out_df['fees'].sum()
print("------outgoing transactions------")
print(f"total amount: {total_out} BTC")
print(f"number of transactions: {n_out_trxs}")
print("")

print("------incoming transactions------")
print(f"total amount: {total_in} BTC")
print(f"number of transactions: {n_in_trxs}")
print("")

print("------wallet history------")
print(f"in transactions: {n_in_trxs}")
print(f"out transactions: {n_out_trxs}")
print(f"total transactions: {n_in_trxs + n_out_trxs}")
print(f"total out: {total_out} BTC")
print(f"total out fees: {total_out_fees} BTC")
print(f"total out with fees: {total_out + total_out_fees} BTC")
print(f"total in: {total_in} BTC")



------outgoing transactions------
total amount: 9.162901309999999 BTC
number of transactions: 478

------incoming transactions------
total amount: 9.28544026 BTC
number of transactions: 1175

------wallet history------
in transactions: 1175
out transactions: 478
total transactions: 1653
total out: 9.162901309999999 BTC
total in: 9.28544026 BTC
